In [11]:
from louisdataset import MyDataSet

In [4]:
def get_DictX(full_dataframe, Y): # e.g. create_dict_list(dataset_train.df)
    
    # temporary fix: can put the variables as arguments
    DictX = full_dataframe.copy()
    del DictX['First Name']
    del DictX['Last Name']
    del DictX['Birthday']
    del DictX['Index'] 
    del DictX[Y]
    return DictX

def get_dummies(full_dataframe, variable):
    Dict = {variable:full_dataframe[variable]}.copy()
    Categories = list(set(Dict[variable])) 
    for i in Categories: 
        Dict[i] = [(element == i)*1 for element in Dict[variable]]
    del Dict[variable]
    return Dict

def get_Y(full_dict, Y):
    return np.array(full_dict[Y].copy())

In [12]:
import pandas as pd
import numpy as np
from scipy import optimize 

In [3]:
def create_dict_list(dict_dict): # on self.df
    newDict = dict.fromkeys(dict_dict.keys())
    for i in dict_dict.keys():
        newDict[i] = list(dict_dict[i].values())
    return newDict

In [5]:
# for a given feature 
def one_hot_encoder(dataframe, column_to_encode) : 
    Encoded_Dict = dataframe.copy()
    NewCategories = list(set(Encoded_Dict[column_to_encode]))
    NewCategories.pop() # remove the last element 
    for i in NewCategories: 
        Encoded_Dict[i] = [(element == i)*1 for element in Encoded_Dict[column_to_encode]]
    del Encoded_Dict[column_to_encode]
    return Encoded_Dict

# for every feature 
def full_one_hot_encoder(dataframe) : # dataframe = NewDict
    keys_str = [keys for keys in dataframe.keys() if type(list(dataframe[keys])[0]) == str]
    Full_Dict = dataframe
    for key in keys_str:
        Full_Dict = one_hot_encoder(Full_Dict, key)
    return np.column_stack(list(Full_Dict.values())) # return the dictionary as a matrix for the LogReg

# TEST

In [ ]:
from louisdataset import MyDataSet

name_Y = 'Hogwarts House'

dataset_train = MyDataSet().read_csv('resources/dataset_train.csv')
NewDict = dataset_train.dict_list()

# getting X
DictX = get_DictX(NewDict, name_Y)
X = full_one_hot_encoder(DictX)

# getting Y
SubDict = get_dummies(NewDict, 'Hogwarts House')
Y = get_Y(SubDict, 'Gryffindor')

# Getting X and Y 

In [6]:
dataset_train = MyDataSet().read_csv('resources/dataset_train.csv')
# dataset_train.describe()

NewDict = create_dict_list(dataset_train.df)
name_Y = 'Hogwarts House'

# getting X
DictX = get_DictX(NewDict, name_Y)
X = full_one_hot_encoder(DictX)

# getting Y
SubDict = get_dummies(NewDict, 'Hogwarts House')
Y = get_Y(SubDict, 'Gryffindor')

In [7]:
print(Y.shape)
print(X.shape)

(1600,)
(1600, 14)


# Loss Function

In [8]:
def neg_loglikelihood(beta, Y, X):
    # sum without NAs
    return -np.nansum(Y*np.matmul(X,beta) - np.log(1+np.exp(np.matmul(X,beta))))

In [9]:
def get_LogReg_coef(Y_train, X_train):
    n, d = X_train.shape
    init_w = np.zeros(d)
    res = optimize.minimize(neg_loglikelihood,init_w, method = 'BFGS', args = (Y_train,X_train))
    return res.x

In [10]:
get_LogReg_coef(Y, X)

/Users/louislimnavong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/louislimnavong/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:643: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/louislimnavong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


array([-6.73873258e-04,  1.94102649e-02,  2.10344212e+00, -3.14179045e-02,
       -1.80698276e+00, -2.46670581e-02,  2.71721871e-01, -2.48479360e+00,
       -1.82458665e-01, -1.67631934e+00,  8.22153813e-02, -4.34301110e-01,
       -3.70778470e-01, -8.27701406e-01])